NLP Assignment-3 : Sentiment Analysis- Amit Kumar Singh Yadav| 16110011

In [52]:


#ALL HEADER FILES INCLUDED
import nltk
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.layers import SpatialDropout1D
from keras.models import Model
from keras.layers import Input,Flatten, Dense, Embedding, RNN, Conv1D, BatchNormalization, MaxPooling1D, Activation, Dropout, concatenate, Lambda
from keras import optimizers
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from keras import optimizers
import pickle as p

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize

from bs4 import BeautifulSoup  
import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


from google.colab import drive
drive.mount('/content/gdrive')

datapath = '/content/gdrive/My Drive/Colab Notebooks/Assignment_3/'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Train and Text File Reading

In [0]:
with open(datapath+"train.txt","r") as traintext:
  train_file=traintext.read()

with open(datapath+"test.txt","r") as testtext:
  test_file=testtext.read()

Defination for PreProcessing Data and Emoji 



In [0]:
# Emoji2Vec USED EMOJIS - FROM 
remove_en=stopwords.words('english')
exception=['not','never', 'no','ever','very','nothing','really','extremely']
emodict = {0:'amusement', 1:'satisfaction', 2:'optimism', 3:'pride in achievement', 4:'contentment',
           5:'anger', 6:'fear', 7:'disgust', 8:'sadness', 9:'contempt'}
#emotion_label = {
#		"fear" : ['😳', '😰', '😨', '😬'],
#		"anger" : ['😱', '😡', '😤', '😒', '😑', '😠'],
#		"disgust" :['😭', '😩', '🤦', '😯', '🤢', '😖'],
#		"sadness" :['😫', '😭', '😞'] ,
#		"contempt" :['😔', '💔', '😏'],
#		"amusement" :['😲'],
#   "pride in achievement" :['👌', '👏', '👍'],
#    "satisfaction" :['😊', '🙏', '😔', '😎'],
#    "optimism" :['💪', '😎', '🛐'],
#    "contentment" :['😇', '😘', '😂', '😁', '💃'],
#    "none": ['none']  
#        }

emoji_label={
    "negative":['😳', '😰', '😨', '😬','😱', '😡', '😤', '😒', '😑', '😠','😭', '😩', '🤦', '😯', '🤢', '😖', '😫', '😭', '😞','😔', '💔', '😏' ],# fear+anger+disgust+sadness+contempt
    "neutral":['😊', '🙏'],#satisfaction
    "positive":['😇', '😘', '😂', '😁', '💃','💪', '😎', '🛐','👌', '👏', '👍','😲']#optimmism + contentment
}
emoji=['😳', '😰', '😨', '😬','😱', '😡', '😤', '😒', '😑', '😠','😭', '😩', '🤦', '😯', '🤢', '😖', '😫', '😭', '😞','😔', '💔', '😏', '😊', '🙏','😇', '😘', '😂', '😁', '💃','💪',
       '😎', '🛐','👌', '👏', '👍','😲']



Importing Data and Pre-Processing

In [5]:
train_id, train_text, train_label, train_c_h, train_c_e, train_c_o=[],[],[],[],[],[]

for tweet in train_file.split('\n\n'):
  lines=tweet.split('\n')
  try:
    train_label.append(lines[0].split()[2])
    train_id.append(lines[0].split()[1])
  except IndexError:
    del train_id[-1]
    continue
  temp_c,temp_c_h,temp_c_e,temp_c_o=[],[],[],[]
  for line in lines[1:]:
    words=line.split('\t')
    words[0]=words[0].lower()
    if(words[1]!='0'):
      words[0]=re.sub('[\W_]+','',words[0])
    if words[1]=='Eng' and words[0] in remove_en and words[0] not in exception:
      continue
    if 'http' in words[0]:
      continue
    temp_c.append(words[0])
    if words[1]== 'Eng':
      temp_c_e.append(words[0])
    if words[1]== 'Hin':
      temp_c_h.append(words[0])
    elif words[1]== 'O':
      if(words[0] in emoji):
        temp_c_o.append(words[0])
      
  if temp_c==[]:
    continue
  train_text.append(temp_c)
  train_c_h.append(temp_c_h)
  train_c_e.append(temp_c_e)
  train_c_o.append(temp_c_o)


print(len(train_text))


15131


Pre-Processing and Importing Test Data

In [6]:
test_id, test_text, test_label, test_c_h, test_c_e, test_c_o=[],[],[],[],[],[]

for tweet in test_file.split('\n\n'):
  lines=tweet.split('\n')
  try:
    test_label.append(lines[0].split()[2])
    test_id.append(lines[0].split()[1])
  except IndexError:
    del test_id[-1]
    continue
  temp_c,temp_c_h,temp_c_e,temp_c_o=[],[],[],[]
  for line in lines[1:]:
    words=line.split('\t')
    words[0]=words[0].lower()
    if(words[1]!='0'):
      #words[0]=re.sub('[\W_]+','',words[0])
      words[0]=words[0]
    if words[1]=='Eng' and words[0] in remove_en and words[0] not in exception:
      continue
    if 'http' in words[0]:
      continue
    temp_c.append(words[0])
    if words[1]== 'Eng':
      temp_c_e.append(words[0])
    if words[1]== 'Hin':
      temp_c_h.append(words[0])
    elif words[1]== 'O':
      if(words[0] in emoji):
        temp_c_o.append(words[0])
  if temp_c==[]:
    continue
  test_text.append(temp_c)
  test_c_h.append(temp_c_h)
  test_c_e.append(temp_c_e)
  test_c_o.append(temp_c_o)


print(len(test_text))


1869


Compiling all Train and Test Data

In [7]:
train_tweet= [' '.join(i) for i in train_text]
test_tweet= [' '.join(i) for i in test_text]

train_tweet_dict, test_tweet_dict={},{}

train_tweet_dict['hin']= [' '.join(i) for i in train_c_h]
train_tweet_dict['eng']= [' '.join(i) for i in train_c_e]

test_tweet_dict['hin']= [' '.join(i) for i in test_c_h]
test_tweet_dict['eng']= [' '.join(i) for i in test_c_e]

train_tweet_dict['o']= [' '.join(i) for i in train_c_o]
test_tweet_dict['o']= [' '.join(i) for i in test_c_o]

np.unique(train_label)

array(['negative', 'neutral', 'positive'], dtype='<U8')

Fitting Tokenizer

In [0]:
max_features = 20000
tokenizer1 = Tokenizer(num_words=max_features)
tokenizer1.fit_on_texts(train_tweet)
max_len = 300
num_classes = 3




Making Embeddings Dictionary for Negative, Positive and Neutral Sentiment

In [0]:
from numpy import *
tweet_ = {}
for label in ['negative', 'neutral', 'positive']:
  tweet_[label] = {}
  for key in ['eng', 'hin', 'o']:
    tweet_[label][key] = []

# negative_tweet_e, positive_tweet_e, neutral_tweet_e=[],[],[]
# negative_tweet_h, positive_tweet_h, neutral_tweet_h=[],[],[]
# negative_tweet_o, positive_tweet_o, neutral_tweet_o=[],[],[]
train_tweet_embeddings=[] #eng+hindi+emoji

for i in range(len(train_label)):
  temp=[]
  for key in ['eng', 'hin', 'o']:
    words = train_tweet_dict[key][i].split(" ")
    seq=tokenizer1.texts_to_sequences(words)
    vector_list=sequence.pad_sequences(seq, maxlen=max_len)
    #vector_list = [model[word] for word in words if word in model.vocab]
    j = 0
    while j < len(vector_list):
      if sum(np.isnan(vector_list[j]))>0:
        del vector_list[j]
      else:
        j += 1
    avg_vector = np.nanmean(vector_list,axis=0)
    if sum(np.isnan(avg_vector)) == 0:
      tweet_[train_label[i]][key].append(avg_vector)
    temp.append(avg_vector)
  temp = np.average(temp,axis=0)
  train_tweet_embeddings.append(temp)



Test Embeddings

In [0]:
test_tweet_embeddings=[]
for i in range(len(test_tweet)):
  words = test_tweet[i].split(" ")
  seq=tokenizer1.texts_to_sequences(words)
  vector_list=sequence.pad_sequences(seq, maxlen=max_len)
  avg_vector = np.average(vector_list,axis=0)
  test_tweet_embeddings.append(avg_vector)


Weighted Averaging embedding for each type

In [14]:
avg_={}
for label in ['negative','neutral','positive']:
  avg_[label]={}
  for key in ['eng','hin','o']:
    avg_[label][key]=np.nanmean(tweet_[label][key],axis=0)
    print(sum(avg_[label][key]))


2420.1217563024325
1574.2338323387705
0.0
2513.426942012764
1745.986567814108
0.0
2246.894273742185
1710.3447755149716
0.0


Creating Similarity Feature for training

In [0]:
import scipy
similarity=[]
for i in range(len(train_tweet_embeddings)):
  temp=[]
  for label in ['negative','neutral','positive']:
    for key in ['eng','hin']:
      distance = scipy.spatial.distance.cosine(avg_[label][key],train_tweet_embeddings[i])
      temp.append(distance)
  similarity.append(temp)

Creating Similarity Feature for Testing

In [0]:
similarity_test=[]
for i in range(len(test_tweet_embeddings)):
  temp=[]
  for label in ['negative','neutral','positive']:
    for key in ['eng','hin']:
      distance = scipy.spatial.distance.cosine(avg_[label][key],test_tweet_embeddings[i])
      temp.append(distance)
  similarity_test.append(temp)

In [17]:
similarity=np.array(similarity)
similarity_test=np.array(similarity_test)

similarity.shape, similarity_test.shape



((15131, 6), (1869, 6))

Creting Y Labels

In [0]:
label_v = {'negative':0, 'neutral':1, 'positive':2}
y_train = np.array([label_v[i] for i in train_label])
y_test = np.array([label_v[i] for i in test_label])
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)


Using Decision Tree

In [22]:
from sklearn.tree import DecisionTreeClassifier

model=DecisionTreeClassifier()
model.fit(similarity,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Accuracy for Decision Tree

In [23]:
preds=model.predict(similarity_test)
preds=np.argmax(preds,axis=1)
np.sum(preds==y_test)/len(y_test)

0.28517924023542

TOkenizing Sentence

In [24]:
sequences_train = tokenizer1.texts_to_sequences(train_tweet)
sequences_test = tokenizer1.texts_to_sequences(test_tweet)
X_train = sequence.pad_sequences(sequences_train, maxlen=max_len)
X_test = sequence.pad_sequences(sequences_test, maxlen=max_len)
X_train_1=np.hstack((X_train, similarity))
X_test_1=np.hstack((X_test, similarity_test))
print('X_train shape:', X_train_1.shape)
print('X_test shape:', X_test_1.shape)

X_train shape: (15131, 306)
X_test shape: (1869, 306)


Training Normal Neural Network

In [26]:
nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


neural = Sequential()
neural.add(Dense(306))
neural.add(Dropout(0.2))
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)

neural.add(Dense(hidden_dims))
neural.add(Dropout(0.2))
neural.add(Activation('relu'))
neural.add(Dense(num_classes))
neural.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
neural.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

Fitting on Neural Network

In [29]:
neural.fit(X_train_1, Y_train, epochs = 4)

Epoch 1/4
15131/15131 [==============================] - 3s 169us/step - loss: nan - acc: 0.2947
Epoch 2/4
15131/15131 [==============================] - 3s 170us/step - loss: nan - acc: 0.2947
Epoch 3/4
15131/15131 [==============================] - 3s 170us/step - loss: nan - acc: 0.2947
Epoch 4/4
15131/15131 [==============================] - 3s 172us/step - loss: nan - acc: 0.2947


Accuracy of Neural Network

In [30]:
pred = neural.predict_classes(X_test_1, verbose=0)
np.sum(pred==y_test)/len(y_test)

0.28517924023542

Using CNN

In [32]:
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)


nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


CNN = Sequential()
CNN.add(Embedding(max_features, 400))
CNN.add(SpatialDropout1D(0.2))
CNN.add(Convolution1D(nb_filter=nb_filter,
                      filter_length=filter_length,
                      border_mode='valid',
                      activation='tanh',
                      subsample_length=1))
CNN.add(Lambda(max_1d, output_shape=(nb_filter,)))
CNN.add(Dense(hidden_dims))
CNN.add(Dropout(0.2))
CNN.add(Activation('relu'))
CNN.add(Dense(num_classes))
CNN.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
CNN.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`


In [33]:
CNN.fit(X_train_1, Y_train, epochs = 2)

Epoch 1/2
15131/15131 [==============================] - 209s 14ms/step - loss: 0.8885 - acc: 0.5667
Epoch 2/2
15131/15131 [==============================] - 217s 14ms/step - loss: 0.6217 - acc: 0.7395


Accuracy for 2 Epochs

In [34]:
predsc1 = CNN.predict_classes(X_test_1, verbose=0)
np.sum(predsc1==y_test)/len(y_test)

0.5575173889780631

In [37]:
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)


nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


CNN2 = Sequential()
CNN2.add(Embedding(max_features, 400))
CNN2.add(SpatialDropout1D(0.2))
CNN2.add(Convolution1D(nb_filter=nb_filter,
                      filter_length=filter_length,
                      border_mode='valid',
                      activation='tanh',
                      subsample_length=1))
CNN2.add(Lambda(max_1d, output_shape=(nb_filter,)))
CNN2.add(Dense(hidden_dims))
CNN2.add(Dropout(0.2))
CNN2.add(Activation('relu'))
CNN2.add(Dense(num_classes))
CNN2.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
CNN2.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="tanh", filters=300, kernel_size=3, strides=1, padding="valid")`


In [0]:
from copy import deepcopy

In [0]:
orig = deepcopy(CNN2)

In [39]:
CNN2.fit(X_train_1, Y_train, epochs = 1)

Epoch 1/1
15131/15131 [==============================] - 214s 14ms/step - loss: 0.8873 - acc: 0.5665


Accuracy of CNN for 1 Epochs

In [41]:
predsc2 = CNN2.predict_classes(X_test_1, verbose=0)
np.sum(predsc2==y_test)/len(y_test)

0.5708935259497058

In [42]:
from sklearn.metrics import precision_recall_fscore_support as p_recall_f
p_recall_f(y_test, predsc2, average='micro')

(0.5708935259497058, 0.5708935259497058, 0.5708935259497058, None)

In [43]:
p_recall_f(y_test, predsc2, average='macro')

(0.5760576474072049, 0.5717489475477215, 0.5736857827882093, None)

In [44]:
p_recall_f(y_test, predsc2, average='weighted')

(0.5720549902693824, 0.5708935259497058, 0.571245124694644, None)

MODEL SUMMARY FOR CNN

In [46]:
CNN2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 400)         8000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, None, 400)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 300)         360300    
_________________________________________________________________
lambda_2 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 300)              

LSTM MODEL Defination

In [53]:
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)


nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


LSTM1 = Sequential()
LSTM1.add(Embedding(max_features, 400, dropout=0.2))
LSTM1.add(LSTM(30, activation='tanh' ))

LSTM1.add(Dense(hidden_dims))
LSTM1.add(Dropout(0.2))
LSTM1.add(Activation('relu'))
LSTM1.add(Dense(num_classes))
LSTM1.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
LSTM1.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  del sys.path[0]


In [54]:
LSTM1.fit(X_train_1, Y_train, epochs = 1)

Epoch 1/1
15131/15131 [==============================] - 221s 15ms/step - loss: 0.9014 - acc: 0.5545


In [70]:
predls1 = LSTM1.predict_classes(X_test_1, verbose=0)
np.sum(predls1==y_test)/len(y_test)

0.5698234349919743

Accuracy for 1 Epoch trained LSTM

In [73]:
p_recall_f(y_test, predls1, average='micro')

(0.5698234349919743, 0.5698234349919743, 0.5698234349919743, None)

In [72]:
p_recall_f(y_test, predls1, average='macro')

(0.5983334917466654, 0.55528980704776, 0.5646979083528513, None)

In [71]:
p_recall_f(y_test, predls1, average='weighted')

(0.5886240195091994, 0.5698234349919743, 0.5673104291822505, None)

LSTM MODEL SUMMARY

In [74]:
LSTM1.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 400)         8000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                51720     
_________________________________________________________________
dense_8 (Dense)              (None, 300)               9300      
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
activation_7 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 903       
_________________________________________________________________
activation_8 (Activation)    (None, 3)                

In [62]:
from keras import optimizers
def max_1d(X):
    return K.max(X, axis=1)


nb_filter = 300
filter_length = 3
hidden_dims = 300 # 250
nb_epoch = 2


LSTM2 = Sequential()
LSTM2.add(Embedding(max_features, 400, dropout=0.2))
LSTM2.add(LSTM(30, activation='tanh' ))

LSTM2.add(Dense(hidden_dims))
LSTM2.add(Dropout(0.2))
LSTM2.add(Activation('relu'))
LSTM2.add(Dense(num_classes))
LSTM2.add(Activation('sigmoid'))
adam = optimizers.Adam(lr=0.001, decay=1e-6)
LSTM2.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  del sys.path[0]


In [0]:
origLSTM = deepcopy(LSTM2)

In [65]:
LSTM2.fit(X_train_1, Y_train, epochs = 3)

Epoch 1/3
15131/15131 [==============================] - 219s 15ms/step - loss: 0.9107 - acc: 0.5444
Epoch 2/3
15131/15131 [==============================] - 217s 14ms/step - loss: 0.6730 - acc: 0.7101
Epoch 3/3
15131/15131 [==============================] - 211s 14ms/step - loss: 0.4315 - acc: 0.8352


In [66]:
predls2 = LSTM2.predict_classes(X_test_1, verbose=0)
np.sum(predls2==y_test)/len(y_test)

0.5377207062600321

In [67]:
p_recall_f(y_test, predls2, average='micro')

(0.5377207062600321, 0.5377207062600321, 0.5377207062600321, None)

In [68]:
p_recall_f(y_test, predls2, average='macro')

(0.5394084152440005, 0.5469916876149264, 0.5415937362161314, None)

In [69]:
p_recall_f(y_test, predls2, average='weighted')

(0.5368771451793373, 0.5377207062600321, 0.5358117538174243, None)